In [1]:
!pip -q install transformers>=4.38.0 evaluate

import evaluate
from datasets import load_dataset
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model
import torch
from tqdm.auto import tqdm
import numpy as np

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.


2025-11-16 05:00:10.062735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763269210.244118      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763269210.297012      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
dataset = load_dataset("cnn_dailymail", "3.0.0")

README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [3]:
checkpoint = 'facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map='auto')

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1614: UserWarning: The following device_map keys do not match any submodules in the model: ['final_logits_bias']
  warnings.warn(


In [4]:
def tokenize_func(samples):
    inputs = tokenizer(
        samples['article'],
        max_length=model.config.max_position_embeddings,
        padding='longest',
        truncation=True,
        return_tensors='pt'
    )

    with tokenizer.as_target_tokenizer():
        outputs = tokenizer(
            samples['highlights'],
            max_length=model.config.max_position_embeddings,
            padding='longest',
            truncation=True,
            return_tensors='pt'
        )

    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': outputs['input_ids']
    }

dataset = dataset.remove_columns('id')

test_data = dataset['test']

dataset['train'] = dataset['train'].select(range(50000))
dataset['validation'] = dataset['validation'].select(range(4000))

dataset = dataset.map(tokenize_func, batched=True, batch_size=1000)
dataset = dataset.remove_columns(['article', 'highlights'])
train_dataset, val_dataset = dataset['train'], dataset['validation']

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [5]:
data_collator = DataCollatorForSeq2Seq(
    model=model,
    tokenizer=tokenizer
)

rouge = evaluate.load('rouge')
def compute_metrics(eval_preds):
    predictions, labels = eval_preds

    if isinstance(predictions, tuple):
        predictions = predictions[0]

    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    return {
        'rouge1': result['rouge1'],
        'rouge2': result['rouge2'],
        'rougeL': result['rougeL'],
        'rougeLsum': result['rougeLsum']
    }

args = Seq2SeqTrainingArguments(
    # gradient_accumulation_steps=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    output_dir='./result',
    report_to='none',
    learning_rate=5e-5,
    lr_scheduler_type='cosine',
    warmup_ratio=0.1,
    logging_strategy='steps',
    eval_strategy='steps',
    save_strategy='steps',
    logging_steps=25,
    eval_steps=25,
    save_steps=100,
    save_total_limit=5,
    load_best_model_at_end=True,
    metric_for_best_model='rouge2',
    greater_is_better=True,
    num_train_epochs=1,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=4,
    predict_with_generate=True,
    generation_max_length=128,
    generation_num_beams=1,
    eval_on_start=True
)
trainer = Seq2SeqTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=train_dataset.select(range(0, 50000, 10)),
    eval_dataset=val_dataset.select(range(0, 4000, 40)),
    args=args,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
# trainer.train()
# в данном случае дообучение вредит, потому что 'facebook/bart-large-cnn' была и так обучена на этом датасете

In [6]:
trainer.model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        

In [7]:
import gc

print(gc.collect())

for i in range(torch.cuda.device_count()):
    with torch.cuda.device(i):
        torch.cuda.empty_cache()

5797


In [8]:
def generate_response(
    model, tokenizer, prompts,
    temperature=0.7, top_p=0.9, repetition_penalty=1.0, 
    length_penalty=1.0, num_beams=4, num_return_sequences=1,
    *args, **kwargs
):
    inputs = tokenizer(
        prompts,
        return_tensors='pt',
        padding=True,
        truncation=True, 
        max_length=512
    ).to(model.device)

    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            *args,
            **inputs,
            max_new_tokens=256,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            repetition_penalty=repetition_penalty,
            length_penalty=length_penalty,
            num_beams=max(num_beams, num_return_sequences),
            num_return_sequences=num_return_sequences,
            **kwargs
        )

    results = []
    for i in range(len(prompts)):
        prompt_results = []
        for j in range(num_return_sequences):
            idx = i * num_return_sequences + j
            text = tokenizer.decode(outputs[idx], skip_special_tokens=True)
            prompt_results.append(text)
        results.append(prompt_results)

    return results

rouge = evaluate.load('rouge')
def evaluate_model(model, tokenizer, val_data, batch_size=8, *args, **kwargs):
    predictions = []

    for i in tqdm(range(0, len(val_data), batch_size)):
        batch = [val_data[j] for j in range(i, min(i + batch_size, len(val_data)))]
        batch_predictions = generate_response(model, tokenizer, [item['article'] for item in batch], *args, **kwargs)
        batch_predictions = np.array(batch_predictions).ravel().tolist()
        predictions.extend(batch_predictions)

    references = [item['highlights'] for item in val_data]

    return rouge.compute(predictions=predictions, references=references), predictions

In [9]:
generated_text = generate_response(
    trainer.model.half(), tokenizer,
    [test_data['article'][0]]
)[0]
print(generated_text)

["The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki said it was a move toward greater justice."]


In [ ]:
scores, preds = evaluate_model(trainer.model.half(), tokenizer, test_data.select(range(0, 4000, 40)), batch_size=4, num_beams=2)

print(scores)
for pred in preds:
    print('\n########################\n')
    print(pred)

In [11]:
# best_score = 0
# best_preds = None
# best_params = None
# for temperature in tqdm([0.7, 0.8, 0.9]):
#     for top_p in tqdm([0.8, 0.87, 0.95], leave=False):
#         for length_penalty in tqdm([1.0, 1.1, 1.2], leave=False):
#             for num_beams in tqdm([4], leave=False):
#                 scores, preds = evaluate_model(trainer.model.half(), tokenizer, test_data.select(range(40)), batch_size=8,
#                                                temperature=temperature, top_p=top_p, length_penalty=length_penalty, num_beams=num_beams)
#                 if best_score < scores['rouge2']:
#                     best_score = scores['rouge2']
#                     best_preds = preds
#                     best_params = {
#                         'temperature': temperature,
#                         'top_p': top_p,
#                         'length_penalty': length_penalty,
#                         'num_beams': num_beams
#                     }
#                     print(f'New best score: {round(best_score, 5)}')
#                     print(f'New best parameters: {best_params}')
#                     print('#######################################')

# print(f'BEST SCORE: {round(best_score, 5)}')
# print(f'BEST PARAMETERS: {best_params}')

In [112]:
import re
import numpy as np
from scipy.optimize import differential_evolution
from scipy.stats import spearmanr
import pickle
from pathlib import Path
from tqdm.auto import tqdm
from collections import Counter
import evaluate

class TextReRankerWeightsOptimizer:
    def __init__(
        self,
        use_rouge_with_source=True,
        use_extractive_coverage=True,
        use_lead3_overlap=False,
        use_length_simple=True,
        use_compression_ratio=False,
        use_novel_ngrams=False,
        
        length_params=None,
        compression_params=None,
        weights=None,
        device='cuda',
        cache_dir='./reranker_cache'
    ):
        self.use_rouge_with_source = use_rouge_with_source
        self.use_extractive_coverage = use_extractive_coverage
        self.use_lead3_overlap = use_lead3_overlap
        self.use_length_simple = use_length_simple
        self.use_compression_ratio = use_compression_ratio
        self.use_novel_ngrams = use_novel_ngrams
        
        self.length_params = length_params or {'target_min': 50, 'target_max': 70}
        self.compression_params = compression_params or {'optimal_ratio': 0.15, 'sigma': 0.05}
        
        self.weights = weights or self._get_default_weights()
        
        self.device = device
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        
        self._rouge = None
    
    def __getstate__(self):
        state = self.__dict__.copy()
        state['_rouge'] = None
        return state
    
    def __setstate__(self, state):
        self.__dict__.update(state)
    
    def _get_default_weights(self):
        active_metrics = []
        
        if self.use_rouge_with_source:
            active_metrics.append('rouge_with_source')
        if self.use_extractive_coverage:
            active_metrics.append('extractive_coverage')
        if self.use_lead3_overlap:
            active_metrics.append('lead3_overlap')
        if self.use_length_simple:
            active_metrics.append('length_simple')
        if self.use_compression_ratio:
            active_metrics.append('compression_ratio')
        if self.use_novel_ngrams:
            active_metrics.append('novel_ngrams')
        
        if not active_metrics:
            return {}
        
        weight = 1.0 / len(active_metrics)
        return {metric: weight for metric in active_metrics}
    
    def _load_rouge(self):
        if self._rouge is None:
            self._rouge = evaluate.load('rouge')
        return self._rouge
    
    @staticmethod
    def _get_ngrams_fast(text, n):
        words = text.lower().split()
        return Counter([' '.join(words[i:i+n]) for i in range(len(words)-n+1)])
    
    def compute_batch(self, candidates, source_text):
        if not candidates:
            return []
        
        source_ngrams = None
        source_bigrams_set = None
        lead3_text = None
        source_len = None
        
        if self.use_extractive_coverage or self.use_novel_ngrams:
            source_ngrams = {
                1: self._get_ngrams_fast(source_text, 1),
                2: self._get_ngrams_fast(source_text, 2),
                3: self._get_ngrams_fast(source_text, 3),
                4: self._get_ngrams_fast(source_text, 4),
            }
        
        if self.use_novel_ngrams:
            source_bigrams_set = set(source_ngrams[2].keys())
        
        if self.use_lead3_overlap:
            sentences = re.split(r'[.!?]+', source_text)
            sentences = [s.strip() for s in sentences if s.strip()]
            lead3_text = ' '.join(sentences[:3]) if len(sentences) >= 3 else source_text
        
        if self.use_compression_ratio:
            source_len = len(source_text.split())
        
        all_scores = {}
        
        if self.use_rouge_with_source:
            rouge = self._load_rouge()
            scores = []
            for candidate in candidates:
                result = rouge.compute(
                    predictions=[candidate],
                    references=[source_text],
                    rouge_types=['rougeL']
                )
                scores.append(result['rougeL'])
            all_scores['rouge_with_source'] = scores
        
        if self.use_extractive_coverage:
            scores = []
            for candidate in candidates:
                ngram_scores = []
                for n in [1, 2, 3, 4]:
                    cand_ngrams = self._get_ngrams_fast(candidate, n)
                    if not cand_ngrams:
                        continue
                    overlap = sum((cand_ngrams & source_ngrams[n]).values())
                    total = sum(cand_ngrams.values())
                    ngram_scores.append(overlap / total)
                scores.append(np.mean(ngram_scores) if ngram_scores else 0.0)
            all_scores['extractive_coverage'] = scores
        
        if self.use_lead3_overlap:
            rouge = self._load_rouge()
            scores = []
            for candidate in candidates:
                result = rouge.compute(
                    predictions=[candidate],
                    references=[lead3_text],
                    rouge_types=['rougeL']
                )
                scores.append(result['rougeL'])
            all_scores['lead3_overlap'] = scores
        
        if self.use_length_simple:
            target_min = self.length_params['target_min']
            target_max = self.length_params['target_max']
            
            scores = []
            for candidate in candidates:
                words = len(candidate.split())
                if target_min <= words <= target_max:
                    score = 1.0
                elif words < target_min:
                    score = words / target_min
                else:
                    score = max(0, 1 - (words - target_max) / (target_max * 0.5))
                scores.append(score)
            all_scores['length_simple'] = scores
        
        if self.use_compression_ratio:
            optimal = self.compression_params['optimal_ratio']
            sigma = self.compression_params['sigma']
            
            scores = []
            for candidate in candidates:
                cand_len = len(candidate.split())
                ratio = cand_len / source_len if source_len > 0 else 0
                score = np.exp(-((ratio - optimal) ** 2) / (2 * sigma ** 2))
                scores.append(score)
            all_scores['compression_ratio'] = scores
        
        if self.use_novel_ngrams:
            scores = []
            for candidate in candidates:
                cand_bigrams = self._get_ngrams_fast(candidate, 2)
                if not cand_bigrams:
                    scores.append(0.0)
                    continue
                novel = sum(count for bigram, count in cand_bigrams.items() 
                           if bigram not in source_bigrams_set)
                total = sum(cand_bigrams.values())
                scores.append(novel / total if total > 0 else 0.0)
            all_scores['novel_ngrams'] = scores
        
        results = []
        for i in range(len(candidates)):
            candidate_scores = {metric: all_scores[metric][i] for metric in all_scores}
            results.append(candidate_scores)
        
        return results
    
    def compute(self, text, source_text=None, return_individual=True, return_weighted=True):
        batch_results = self.compute_batch([text], source_text)
        scores = batch_results[0]
        
        result = {}
        if return_individual:
            result['scores'] = scores
        
        if return_weighted:
            weighted_score = sum(scores[m] * self.weights.get(m, 0) for m in scores)
            result['weighted_score'] = weighted_score
        
        return result
    
    def rank_candidates(self, candidates, source_text=None):
        if not candidates:
            return []
        
        batch_scores = self.compute_batch(candidates, source_text)
        
        results = []
        for idx, scores in enumerate(batch_scores):
            weighted_score = sum(scores[m] * self.weights.get(m, 0) for m in scores)
            results.append((idx, weighted_score, scores))
        
        results.sort(key=lambda x: x[1], reverse=True)
        
        return results
    
    def get_best_candidate(self, candidates, source_text=None):
        results = self.rank_candidates(candidates, source_text)
        best_idx = results[0][0]
        return candidates[best_idx]
    
    def fit(
        self,
        candidates_list,
        x_texts,
        y_texts,
        metric='rouge2',
        cache_name=None,
        use_cache=True,
        max_iter=50,
        popsize=15,
        seed=42,
        n_workers=1,
        print_correlations=False
    ):
        if len(candidates_list) != len(x_texts) or len(candidates_list) != len(y_texts):
            raise ValueError(
                f"Length mismatch: candidates_list={len(candidates_list)}, "
                f"x_texts={len(x_texts)}, y_texts={len(y_texts)}"
            )
        
        precomputed_metrics = self._load_or_compute_metrics(
            candidates_list, x_texts, cache_name, use_cache
        )
        
        print("Предвычисление целевой метрики для всех кандидатов...")
        target_scores_cache = self._precompute_target_scores(
            candidates_list, y_texts, metric
        )
        
        metric_names = list(precomputed_metrics[0][0].keys())
        n_metrics = len(metric_names)

        print(f"\nМетрики: {metric_names}")
        print(f"Целевая метрика: {metric}")
        
        if print_correlations:            
            self._print_correlations_fast(
                precomputed_metrics, target_scores_cache,
                metric_names, n_samples=30
            )
        
        best_weights, best_score = self._optimize_weights_fast(
            candidates_list, precomputed_metrics, target_scores_cache,
            metric_names, max_iter, popsize, seed, n_workers
        )
        
        self.weights = {
            metric_names[i]: best_weights[i]
            for i in range(n_metrics)
        }

        print("\n" + "="*60)
        print("РЕЗУЛЬТАТЫ КАЛИБРОВКИ")
        print("="*60)
        print(f"Best {metric}: {best_score:.4f}")
        print(f"\nОптимальные веса:")
        for name, weight in self.weights.items():
            print(f"  {name:20s}: {weight:.4f}")
        print("="*60)

        return self.weights, best_score
    
    def _load_or_compute_metrics(self, candidates_list, x_texts, cache_name, use_cache):
        cache_path = None
        if cache_name and use_cache:
            cache_path = self.cache_dir / f"{cache_name}_metrics.pkl"
        
        if cache_path and cache_path.exists():
            print(f"Загрузка метрик из кэша: {cache_path}")
            with open(cache_path, 'rb') as f:
                return pickle.load(f)
        
        print("Предвычисление метрик...")
        precomputed = []
        
        for candidates, x_text in tqdm(list(zip(candidates_list, x_texts))):
            candidate_metrics = self.compute_batch(candidates, x_text)
            precomputed.append(candidate_metrics)
        
        if cache_path:
            with open(cache_path, 'wb') as f:
                pickle.dump(precomputed, f)
            print(f"Метрики сохранены в кэш: {cache_path}")
        
        return precomputed
    
    def _precompute_target_scores(self, candidates_list, y_texts, metric='rouge2'):
        rouge = self._load_rouge()
        
        target_scores = []
        
        for candidates, y_text in tqdm(zip(candidates_list, y_texts), 
                                       total=len(candidates_list),
                                       desc="Целевая метрика"):
            candidate_scores = []
            for candidate in candidates:
                result = rouge.compute(
                    predictions=[candidate],
                    references=[y_text]
                )
                candidate_scores.append(result[metric])
            
            target_scores.append(candidate_scores)
        
        return target_scores
    
    def _print_correlations_fast(self, precomputed_metrics, target_scores_cache,
                                 metric_names, n_samples=30):
        print("\nКорреляция с целевой метрикой:")
        
        metric_spearman = {name: [] for name in metric_names}
        
        for i in range(min(n_samples, len(precomputed_metrics))):
            target_values = target_scores_cache[i]
            
            for metric_name in metric_names:
                metric_values = [
                    precomputed_metrics[i][j][metric_name] 
                    for j in range(len(precomputed_metrics[i]))
                ]
                
                if np.std(metric_values) > 1e-10 and np.std(target_values) > 1e-10:
                    spearman, _ = spearmanr(metric_values, target_values)
                    metric_spearman[metric_name].append(spearman)
        
        for metric_name in metric_names:
            if metric_spearman[metric_name]:
                avg_spearman = np.mean(metric_spearman[metric_name])
                std_spearman = np.std(metric_spearman[metric_name])
                
                if abs(avg_spearman) > 0.4:
                    status = "✓✓✓"
                elif abs(avg_spearman) > 0.25:
                    status = "✓✓"
                elif abs(avg_spearman) > 0.15:
                    status = "✓"
                else:
                    status = "⚠️"
                
                print(f"  {status} {metric_name:25s}: {avg_spearman:+.3f} (±{std_spearman:.3f})")
    
    def _optimize_weights_fast(self, candidates_list, precomputed_metrics, 
                               target_scores_cache, metric_names, 
                               max_iter, popsize, seed, n_workers):
        n_metrics = len(metric_names)
        
        self._opt_precomputed_metrics = precomputed_metrics
        self._opt_target_scores_cache = target_scores_cache
        self._opt_metric_names = metric_names
        self._opt_n_metrics = n_metrics
        
        print(f"\nОптимизация весов (workers={n_workers})...")
        
        result = differential_evolution(
            self._objective_function_fast,
            bounds=[(0, 1)] * n_metrics,
            strategy='best1bin',
            maxiter=max_iter,
            popsize=popsize,
            tol=0.001,
            mutation=(0.5, 1.5),
            recombination=0.7,
            seed=seed,
            workers=n_workers,
            updating='deferred' if n_workers > 1 else 'immediate',
            polish=True,
            disp=True
        )
        
        del self._opt_precomputed_metrics
        del self._opt_target_scores_cache
        del self._opt_metric_names
        del self._opt_n_metrics
        
        weights_raw = result.x
        weights_sum = sum(weights_raw)
        
        if weights_sum < 1e-10:
            print("⚠️ WARNING: Все веса близки к нулю, используем равные веса")
            weights_normalized = np.ones(n_metrics) / n_metrics
        else:
            weights_normalized = weights_raw / weights_sum
        
        best_score = -result.fun
        
        return weights_normalized, best_score
    
    def _objective_function_fast(self, weights):
        total_score = 0.0
        
        for metrics_list, target_scores in zip(
            self._opt_precomputed_metrics, 
            self._opt_target_scores_cache
        ):
            weighted_scores = [
                sum(weights[i] * metrics_dict.get(self._opt_metric_names[i], 0) 
                    for i in range(self._opt_n_metrics))
                for metrics_dict in metrics_list
            ]
            
            best_idx = np.argmax(weighted_scores)
            total_score += target_scores[best_idx]
        
        avg_score = total_score / len(self._opt_precomputed_metrics)
        
        return -avg_score

In [113]:
def predict_with_reranker(promts):
    results = []
    for promt in promts:
        generated_texts = generate_response(
            trainer.model.half(), 
            tokenizer,
            [promt],
            num_return_sequences=20,
            num_beams=2,
            temperature=1.0,
            top_k=1000,
            top_p=1.0
        )[0]
        result = reranker.get_best_candidate(generated_texts, promt)
        results.append(result)
    return results

def evaluate_model_with_reranker(model, tokenizer, val_data, batch_size=8):
    predictions = []

    for i in tqdm(range(0, len(val_data), batch_size)):
        batch = [val_data[j] for j in range(i, min(i + batch_size, len(val_data)))]
        batch_predictions = predict_with_reranker([item['article'] for item in batch])
        predictions.extend(batch_predictions)

    references = [item['highlights'] for item in val_data]

    rouge = evaluate.load('rouge')
    final_scores = rouge.compute(predictions=predictions, references=references)
    
    return final_scores, predictions

In [ ]:
reranker = TextReRanker(
    # use_rouge_with_source=True,
    use_rouge_with_source=True,

    use_extractive_coverage=True,

    # use_lead3_overlap=True,
    use_lead3_overlap=True,
    
    use_length_simple=True,

    # use_compression_ratio=True,
    use_compression_ratio=True,

    use_novel_ngrams=True,
    length_params={'target_min': 50, 'target_max': 70},
    compression_params={'optimal_ratio': 0.15, 'sigma': 0.05},

    weights=None,
    device='cuda',
    cache_dir='./reranker_cache'
)

# dev_candidates = []
# dev_x_texts = []
# dev_y_texts = []

# for sample in tqdm(test_data.select(range(0, 4000, 10))):
#     candidates = generate_response(
#         trainer.model.half(), tokenizer, [sample['article']],
#         num_return_sequences=20,
#         num_beams=2,
#         temperature=1.0,
#         top_k=1000,
#         top_p=1.0
#     )[0]

#     dev_candidates.append(candidates)
#     dev_x_texts.append(sample['article'])
#     dev_y_texts.append(sample['highlights'])

optimal_weights, best_score = reranker.fit(
    candidates_list=dev_candidates,
    x_texts=dev_x_texts,
    y_texts=dev_y_texts,
    metric='rouge2',
    cache_name='cnn_dev_v11',
    max_iter=500,
    popsize=30,
    print_correlations=True
)

Предвычисление метрик...


  0%|          | 0/400 [00:00<?, ?it/s]

Метрики сохранены в кэш: reranker_cache/cnn_dev_v11_metrics.pkl
Предвычисление целевой метрики для всех кандидатов...


Целевая метрика:   0%|          | 0/400 [00:00<?, ?it/s]


Метрики: ['rouge_with_source', 'extractive_coverage', 'lead3_overlap', 'length_simple', 'compression_ratio', 'novel_ngrams']
Целевая метрика: rouge2

Корреляция с целевой метрикой:
  ✓✓ rouge_with_source        : -0.312 (±0.415)
  ⚠️ extractive_coverage      : -0.109 (±0.420)
  ⚠️ lead3_overlap            : +0.095 (±0.435)
  ✓✓ length_simple            : -0.324 (±0.442)
  ⚠️ compression_ratio        : -0.044 (±0.569)
  ⚠️ novel_ngrams             : +0.087 (±0.411)

Оптимизация весов (workers=1)...
differential_evolution step 1: f(x)= -0.1914528593882406
differential_evolution step 2: f(x)= -0.1914528593882406
differential_evolution step 3: f(x)= -0.19235395736710953
differential_evolution step 4: f(x)= -0.19293848073812575
differential_evolution step 5: f(x)= -0.19293848073812575
differential_evolution step 6: f(x)= -0.19293848073812575
differential_evolution step 7: f(x)= -0.19293848073812575
differential_evolution step 8: f(x)= -0.19299278617445162
differential_evolution step 9: f(x

In [127]:
def evaluate_on_cached_candidates(dev_candidates, dev_x_texts, dev_y_texts):
    predictions = []
    
    for candidates, x_text in tqdm(list(zip(dev_candidates, dev_x_texts))):
        best = reranker.get_best_candidate(candidates, x_text)
        predictions.append(best)
    
    rouge = evaluate.load('rouge')
    scores = rouge.compute(predictions=predictions, references=dev_y_texts)
    
    return scores, predictions

# использование
scores, preds = evaluate_on_cached_candidates(dev_candidates, dev_x_texts, dev_y_texts)
print(scores)  # должно совпасть с fit()

  0%|          | 0/400 [00:00<?, ?it/s]

{'rouge1': 0.40129553424797715, 'rouge2': 0.186614049793906, 'rougeL': 0.28040691109978166, 'rougeLsum': 0.33999917830156656}


In [115]:
scores, preds = evaluate_model_with_reranker(trainer.model.half(), tokenizer, test_data.select(range(0, 4000, 10)), batch_size=1)

print(scores)
for pred in preds:
    print('\n########################\n')
    print(pred)

  0%|          | 0/100 [00:00<?, ?it/s]

{'rouge1': 0.4157587599396884, 'rouge2': 0.205582280885957, 'rougeL': 0.30087225115877336, 'rougeLsum': 0.35695923131441254}

########################

The Palestinian Authority becomes the 123rd member of the International Criminal Court. The formal accession was marked with a ceremony at The Hague, in the Netherlands. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States had opposed the Palestinians' efforts to join.

########################

The temperature was recorded at Argentina's Esperanza Base on the northern tip of the Antarctica Peninsula. The World Meteorological Organization is in the process of setting up an international ad-hoc committee. Researchers watch weather changes in the Antarctic region and elsewhere for evidence of climate change.

########################

Ted Cruz drew large crowds during his two-day swing across Iowa last week. He's counting on Iowa, known for its vocal and active evangelical base